In [9]:
import sbol2
from sbol2 import *
import plateo
import plateo.containers
import plateo.tools
import warnings
import pandas as pd
import uuid
from typing import List, Dict
from sbolParserApi import ParserSBOL
import numpy as np
import copy
from rdflib import URIRef
import posixpath
import time

In [146]:
def enumerator(
    doc: sbol2.document.Document,
    derivation: sbol2.combinatorialderivation.CombinatorialDerivation
) -> List[sbol2.componentdefinition.ComponentDefinition]:
    parents = []
    template = doc.getComponentDefinition(derivation.masterTemplate)
    templateCopy = createTemplateCopy(template, template.displayId + "_Var", "1")
    parents.append(templateCopy)
    for vc in derivation.variableComponents:
        newParents = []
        for parent in parents:
            for children in group(collectVariants(doc, vc), "http://sbols.org/v2#one"):
                varDisplayId = concatenateChildrenDisplayId(children)
                start = time.process_time()

                # if not doc.find(parent.persistentIdentity + "_" + varDisplayId + "/1"):
                if parent.persistentIdentity + "_" + varDisplayId + "/1" not in doc.componentDefinitions:
                    # Create parent copy
                    uniqueId = getUniqueDisplayId(None, None, parent.displayId + "_" + varDisplayId, parent.version, "CD", doc)
                    newParent = createTemplateCopy(parent, uniqueId, "1")
                    doc.add(newParent)
                    print(uniqueId, "set:", time.process_time() - start)
                else:
                    # Set newParent to existing CD
                    newParent = doc.getComponentDefinition(parent.persistentIdentity + "_" + varDisplayId + "/1")
                    print(newParent.displayId, "set:", time.process_time() - start)

                # # Create parent copy
                # uniqueId = getUniqueDisplayId(None, None, parent.displayId, parent.version, "CD", doc)
                # newParent = createTemplateCopy(parent, uniqueId, "1")
                # doc.add(newParent)
                # print("added:", newParent.displayId)
                # print(uniqueId, "set:", time.process_time() - start)

                # Add children
                addChildren(template, template.components[vc.variable], newParent, children)
                # Add to newParents
                newParents.append(newParent)
        parents = newParents
    return parents

def addChildren(
    originalTemplate: sbol2.componentdefinition.ComponentDefinition,
    originalComponent: sbol2.component.Component,
    newParent: sbol2.componentdefinition.ComponentDefinition,
    children: List[sbol2.componentdefinition.ComponentDefinition]
):
    newComponent = newParent.components[originalComponent.displayId]
    newComponent.wasDerivedFrom = originalComponent.identity
    if children is None:
        removeConstraintReferences(newParent, newComponent)
        for sa in newParent.sequenceAnnotations:
            if sa.component is not None and sa.component == newComponent.identity:
                newParent.sequenceAnnotations.remove(sa.identity)
        newParent.components.remove(newComponent.identity)
        return
    first = True
    for child in children:
        if first:
            # Take over definition of newParent's version of original component
            newComponent.definition = child.identity
            first = False
        else:
            # Create new component
            uniqueId = getUniqueDisplayId(newParent, None, child.displayId + "_Component", "1", "Component", None)
            link = newParent.components.create(uniqueId)
            link.definition = child.persistentIdentity
            link.access = SBOL_ACCESS_PUBLIC
            link.version = child.version
            link.wasDerivedFrom = originalComponent.identity
            # Create a new 'prev precedes link' constraint
            if originalTemplate.hasUpstreamComponent(originalComponent):
                oldPrev = originalTemplate.getUpstreamComponent(originalComponent)
                if oldPrev.identity in newParent.components:
                    newPrev = newParent.components[oldPrev.identity]
                    uniqueId = getUniqueDisplayId(newParent, None, newParent.displayId + "_SequenceConstraint", None, "SequenceConstraint", None)
                    newSequenceConstraint = newParent.sequenceConstraints.create(uniqueId)
                    newSequenceConstraint.subject = newPrev.identity
                    newSequenceConstraint.object = link.identity
                    newSequenceConstraint.restriction = SBOL_RESTRICTION_PRECEDES
            # Create new 'link precedes next' constraint
            if originalTemplate.hasDownstreamComponent(originalComponent):
                oldNext = originalTemplate.getDownstreamComponent(originalComponent)
                if oldNext.identity in newParent.components:
                    newNext = newParent.components[oldNext.identity]
                    uniqueId = getUniqueDisplayId(newParent, None, newParent.displayId + "_SeqeunceConstraint", None, "SequenceConstraint", None)
                    newSequenceConstraint = newParent.sequenceConstraints.create(uniqueId)
                    newSequenceConstraint.subject = link.identity
                    newSequenceConstraint.object = newNext.identity
                    newSequenceConstraint.restriction = SBOL_RESTRICTION_PRECEDES

def removeConstraintReferences(
    newParent: sbol2.componentdefinition.ComponentDefinition,
    newComponent: sbol2.component.Component
):
    subj = None
    obj = None
    for sc in newParent.sequenceConstraints:
        if sc.subject == newComponent.identity:
            obj = newParent.components[sc.object]
            if subj is not None:
                sc.subject = subj
                obj = None
                subj = None
            else:
                newParent.sequenceConstraints.remove(sc.identity)
        if sc.object == newComponent.identity:
            subj = newParent.components[sc.subject]
            if obj is not None:
                sc.object = obj
                obj = None
                subj = None
            else:
                newParent.sequenceConstraints.remove(sc.identity)

def createTemplateCopy(
    template: sbol2.componentdefinition.ComponentDefinition,
    displayId: str,
    version: str
) -> sbol2.componentdefinition.ComponentDefinition:
    # FIXME: Recreate sbolobject instead of deepcopy
    start = time.process_time()
    newPersistentId = URIRef(posixpath.join(sbol2.getHomespace(), displayId))
    newIdentity = URIRef(posixpath.join(newPersistentId, version))
    newDisplayId = URIRef(displayId)
    # objCopy = copy.copy(obj)
    # objCopy.displayId = newDisplayId
    # objCopy.version = version
    # objCopy.persistentIdentity = newPersistentId
    # objCopy.identity = newIdentity
    templateCopy = sbol2.componentdefinition.ComponentDefinition(newDisplayId, template.types, version)
    templateCopy.roles = template.roles
    # templateCopy.version = version
    # templateCopy.types = template.types
    # TODO: Create components
    primaryStructure = template.getPrimaryStructureComponents()
    curr = None
    prev = None
    for c in primaryStructure:
        curr = templateCopy.components.create(c.displayId)
        curr.access = c.access
        curr.definition = c.definition
        if prev is not None:
            uniqueId = getUniqueDisplayId(templateCopy, None, templateCopy.displayId + "_SequenceConstraint", None, "SequenceConstraint", None)
            sc = templateCopy.sequenceConstraints.create(uniqueId)
            sc.subject = prev.identity
            sc.object = curr.identity
            sc.restriction = SBOL_RESTRICTION_PRECEDES
        prev = curr
    templateCopy.wasDerivedFrom = [template.identity, derivation.identity]
    for c in templateCopy.components:
        component = template.components[c.displayId]
        c.wasDerivedFrom = component.identity
    print(displayId, "copy: ", time.process_time() - start)
    return templateCopy

def getUniqueDisplayId(
    comp: sbol2.componentdefinition.ComponentDefinition = None,
    derivation: sbol2.combinatorialderivation.CombinatorialDerivation = None,
    displayId: str = None,
    version: str = None,
    dataType: str = None,
    doc: sbol2.document.Document = None
) -> str:
    i = 1
    if dataType == "CD":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        # while doc.find(uniqueUri):
        while uniqueUri in [cd.displayId for cd in doc.componentDefinitions]:
            i += 1
            uniqueUri = sbol2.getHomespace() + "%s_%d/%s" %(displayId, i, version)
        if i == 1:
            return displayId
        else:
            return displayId + "_" + str(i)
    elif dataType == "SequenceAnnotation":
        while displayId in [sa.displayId for sa in comp.sequenceAnnotations]:
            i += 1
            displayId = displayId + str(i)
        if i == 1:
            return displayId
        else:
            return displayId
    elif dataType == "SequenceConstraint":
        while displayId in [sc.displayId for sc in comp.sequenceConstraints]:
            i += 1
            displayId = displayId + str(i)
        if i == 1:
            return displayId            
        else:
            return displayId
    elif dataType == "Component":
        while displayId in [c.displayId for c in comp.components]:
            i += 1
            displayId = displayId + str(i)
        if i == 1:
            return displayId
        else:
            return displayId
    elif dataType == "Sequence":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        while doc.find(uniqueUri):
            i += 1
            uniqueUri = sbol2.getHomespace() + "%s_%d/%s" %(displayId, i, version)
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    # TODO: Range
    elif dataType == "CombinatorialDerivation":
        uniqueUri = sbol2.getHomespace() + displayId + "/" + version
        while doc.find(uniqueUri):
            i += 1
            uniqueUri = sbol2.getHomespace() + "%s_%d/%s" %(displayId, i, version)
        if i == 1:
            return displayId
        else:
            return displayId + str(i)
    elif dataType == "VariableComponent":
        while displayId + str(i) in [vc.displayId for vc in derivation.variableComponents]:
            i += 1
            displayId = displayId + str(i)
        if i == 1:
            return displayId
        else:
            return displayId
    else:
        raise ValueError("")

def concatenateChildrenDisplayId(
    children: List[sbol2.componentdefinition.ComponentDefinition]
) -> str:
    concDisplayId = ""
    for child in children:
        concDisplayId = concDisplayId + child.displayId
    return concDisplayId

def collectVariants(
    doc: sbol2.document.Document,
    vc: sbol2.combinatorialderivation.VariableComponent
) -> List[sbol2.componentdefinition.ComponentDefinition]:
    variants = []
    # Add all variants
    for v in vc.variants:
        variant = doc.componentDefinitions.get(v)
        variants.append(variant)
    # Add all variants from Variant Collections
    for c in vc.variantCollections:
        for m in c.members:
            tl = doc.get(m)
            if type(tl) == sbol2.componentdefinition.ComponentDefinition:
                variants.add(tl)
    for derivation in vc.variantDerivations:
        variants.extend(enumerator(doc, doc.get(derivation)))
    return variants

def group(
    variants: List[sbol2.componentdefinition.ComponentDefinition],
    repeat: str
) -> List[List[sbol2.componentdefinition.ComponentDefinition]]:
    groups = []
    for cd in variants:
        group = []
        group.append(cd)
        groups.append(group)
    if repeat == "http://sbols.org/v2#one":
        return groups
    if repeat == "http://sbols.org/v2#zeroOrOne":
        groups.append([])
        return groups
    groups.clear()
    generateCombinations(groups, variants, 0, [])
    if repeat == "http://sbols.org/v2#oneOrMore":
        return groups
    if repeat == "http://sbols.org/v2#zeroOrMore":
        groups.append([])
        return groups

def generateCombinations(
    groups: List[List[sbol2.componentdefinition.ComponentDefinition]],
    variants: List[sbol2.componentdefinition.ComponentDefinition],
    i: int,
    sets: List[sbol2.componentdefinition.ComponentDefinition]
):
    if i == len(variants):
        if not sets:
            groups.add(sets)
        return
    no = sets.copy()
    generateCombinations(groups, variants, i+1, no)
    yes = sets.copy()
    yes.add(variants[i])
    generateCombinations(groups, variants, i+1, yes)


In [147]:
filepath = "../examples/sbol/iGEM2020/Trp_Optimization.xml"
doc = sbol2.document.Document(filepath)
sbol2.setHomespace('http://www.dummy.org/')
sbol2.Config.setOption('sbol_compliant_uris', True)
sbol2.Config.setOption('sbol_typed_uris', True)
parser = ParserSBOL(doc)
derivation = doc.combinatorialderivations.get("http://www.dummy.org/Trp_Optimization_CombinatorialDerivation/1")
l = enumerator(doc, derivation)
for lc in l:
    print(lc.displayId)
    for c in lc.components:
        de = doc.get(c.definition)
        print(de.displayId)

1
TU_dCas9_Var_pCCW12
TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA
Trp_Optimization_Var_TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA_TU_aro3_Var_pPGK1_TU_Tkl1_Var_pREV1_TU_dCas9_Var_pPGK1
TU_Tkl1_Var_pREV1
TU_aro3_Var_pPGK1
TU_dCas9_Var_pPGK1
TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA
Trp_Optimization_Var_TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA_TU_aro3_Var_pPGK1_TU_Tkl1_Var_pREV1_TU_dCas9_Var_pPOP6
TU_Tkl1_Var_pREV1
TU_aro3_Var_pPGK1
TU_dCas9_Var_pPOP6
TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA
Trp_Optimization_Var_TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA_TU_aro3_Var_pPGK1_TU_Tkl1_Var_pREV1_TU_dCas9_Var_pREV1
TU_Tkl1_Var_pREV1
TU_aro3_Var_pPGK1
TU_dCas9_Var_pREV1
TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA
Trp_Optimization_Var_TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA_TU_aro3_Var_pPGK1_TU_Tkl1_Var_pREV1_TU_dCas9_Var_pALD6
TU_Tkl1_Var_pREV1
TU_aro3_Var_pPGK1
TU_dCas9_Var_pALD6
TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA
Trp_Optimization_Var_TU_gRNA_Cassette_Var_PheA5_TyrA1_gRNA_TU_aro3_Var_pPGK1_TU_Tkl1_Var_pREV1_TU_dCas9_Var_pHHF